# BerryBox Berry Segmentation

## Run inferencing using a trained YOLOv8 model

This notebook is meant to be run on a Windows PC using a CPU

In [1]:
# Open ultralytics
import ultralytics
ultralytics.checks()

# Set project directory and change directory
import os
import shutil
proj_dir = os.getcwd()
proj_dir
%ls

# Model inference size
image_size = (1856, 2784)

Ultralytics YOLOv8.0.186  Python-3.9.18 torch-2.2.1+cpu CPU (Intel Core(TM) i7-10810U 1.10GHz)
Setup complete  (12 CPUs, 31.6 GB RAM, 421.1/951.3 GB disk)


 Volume in drive C is OS
 Volume Serial Number is 5E84-0237

 Directory of c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging

05/10/2024  11:28 AM    <DIR>          .
05/10/2024  11:28 AM    <DIR>          ..
03/17/2024  03:42 PM    <DIR>          .ipynb_checkpoints
05/10/2024  11:24 AM    <DIR>          __pycache__
12/29/2022  11:40 AM             2,696 data_cleanup.R
05/10/2024  11:42 AM            14,183 functions.py
03/17/2024  05:56 PM    <DIR>          images
05/10/2024  11:42 AM            21,322 inference.ipynb
05/10/2024  11:24 AM    <DIR>          models
05/10/2024  11:32 AM    <DIR>          output
03/19/2024  11:40 AM    <DIR>          output1
11/22/2022  10:27 AM               464 REAMDE.txt
03/17/2024  04:41 PM                79 requirements.txt
               5 File(s)         38,744 bytes
               8 Dir(s)  569,249,898,496 bytes free


1. Import a YOLO model and export it using OpenVINO

In [2]:
# Set the path to the model
model_path = os.path.join(proj_dir, "models/berrybox_inst_seg_nano_best_20240324.pt")
# model_path = "C:/Users/jeffrey.neyhart/Documents/Phenomics/berryboxai/models/berrybox_inst_seg_nano_best_20240324.pt"
# OpenVino model path
ov_model_path = model_path.replace(".pt", "_openvino_model")

# Attempt to find the openvino version of the model;
# If it does not exist, export the model
if not os.path.exists(ov_model_path):

    os.chdir(os.path.dirname(model_path))

    # Copy the model to the OpenVino location 
    model_path_local = model_path

    # Load the model with YOLO
    from ultralytics import YOLO
    model = YOLO(model_path_local)

    # Export the model using openVINO
    model.export(format = "openvino", imgsz = image_size, half = True)

    os.chdir(proj_dir)

2. Import dependecies, set parameters
Note: Make sure functions.py is in the current working directory

In [3]:
from functions import * # load all functions
from ultralytics import YOLO
import os
import torch
import gc
import shutil
from PIL import Image

gc.collect()   # collect garbage

device = '0' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


"""
------------------------------------------------------------------------------------
Set Directories
------------------------------------------------------------------------------------
"""
model_dir = ov_model_path # path to the model
image_dir = 'images' # path to the image folder
save_dir = 'output' # path to save the results

# shutil.rmtree(save_dir, ignore_errors=True)

"""
------------------------------------------------------------------------------------
Set Model Parameters (you can change these parameters to suit your needs)
------------------------------------------------------------------------------------
"""
model_params = {
    'project': save_dir, # project name
    'name': "berrybox_" + os.path.basename(proj_dir), # run name
    'save': False, # save image results
    'show_labels': True,   # hide labels
    'show_conf': True, # hide confidences
    'save_crop': False, # save cropped prediction boxes
    'line_width': 3, # bounding box line width
    'conf': 0.70, # confidence threshold
    'iou': 0.75, # NMS IoU threshold
    'imgsz': image_size,
    'exist_ok': True, # if True, it overwrites current 'name' saving folder
    'half': True, # use FP16 half-precision inference True/False
    'cache': False, # use cache images for faster inference
    'retina_masks': False, #use high resolution seg mask
    'device': device, # cuda device, i.e. 0 or 0,1,2,3 or cpu
    'verbose': True
}

# Load the model
model = YOLO(model_dir, task = "segment")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Using device: cpu


3. Run Inference

In [4]:
# Create an empty pandas data frame
DF = pd.DataFrame()

# List images in the image dir
image_files = [x for x in os.listdir(image_dir) if ".JPG" in x.upper() or ".PNG" in x.upper()]

# Print the number of images
print('Running inference and extracting features for ' + str(len(image_files)) + " images...\n")

# Target model size
newH, newW = model_params["imgsz"]

# Iterate over the image files
for i, img_name in enumerate(image_files):
    
    # Read in the image and resize
    image = Image.open(image_dir + "/" + img_name).resize((newW, newH))
    
    # Run through the model
    results = model.predict(source = image, **model_params)
    result = results[0]

    # Process the results
    # Try color correction; skip if it doesn't work
    try:
        result, patch_size = color_correction(result)
    except:
        continue
    # Was "info" found?
    if any(result.boxes.cls == get_ids(result, 'info')[0]):
        QR_info = read_QR_code(result)
    else:
        print("No 'info' detected by the model.\n")
        QR_info = img_name
    # Get features
    df1 = get_all_features_parallel(result, name= 'berry')
    df2 = get_all_features_parallel(result, name= 'rotten')
    df = pd.concat([pd.DataFrame({'name': (['berry'] * df1.shape[0]) + (['rotten'] * df2.shape[0])}), pd.concat([df1, df2], ignore_index = True)], axis = 1)    
    w,_ = df.shape
    img_name = [img_name]*w
    QR_info = [QR_info]*w
    patch_size = [np.mean(patch_size)]*w
    indeces = list(range(w))
    # If indeces is length 0; warn that no berries were found
    if len(indeces) == 0:
        print('\033[1;33mNo berries were found in the image!\033[0m')
        continue

    df_fore = pd.DataFrame({'Image_name': img_name,
                            'ID': indeces,
                            'QR_info': QR_info,
                            'Patch_size': patch_size})

    df = pd.concat([df_fore, df], axis=1)
    DF = pd.concat([DF, df], axis=0, ignore_index=True)

    img_save_folder = os.path.join(save_dir, 'predictions')
    if not os.path.exists(img_save_folder):
        os.makedirs(img_save_folder)

    save_ROI_parallel(result, get_ids(result, 'berry'), os.path.join(img_save_folder, img_name[0]))

    print(f"\nImage {i+1} of {len(image_files)} processed." )
    
    
DF.to_csv(os.path.join(save_dir, 'features.csv'), index=False)
print('Done.')

gc.collect()    

Running inference and extracting features for 2 images...



Loading c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2021\Imaging\BerryBoxImaging\models\berrybox_inst_seg_nano_best_20240324_openvino_model for OpenVINO inference...

0: 1856x2784 1 ColorCard, 41 berrys, 1 info, 4117.8ms
Speed: 126.7ms preprocess, 4117.8ms inference, 3934.4ms postprocess per image at shape (1, 3, 1856, 2784)



Color correcting the image...

Initial mean DeltaE:  7.70943872417
Number of components:  10

Color correction complete.


Reading QR code...


QR code reading complete.



Extracting features: 100%|██████████| 41/41 [00:11<00:00,  3.44it/s]
Extracting features: 0it [00:00, ?it/s]



Image 1 of 2 processed.



0: 1856x2784 1 ColorCard, 41 berrys, 1 info, 994.7ms
Speed: 76.7ms preprocess, 994.7ms inference, 2586.0ms postprocess per image at shape (1, 3, 1856, 2784)



Color correcting the image...

Initial mean DeltaE:  7.39397356979
Number of components:  10

Color correction complete.


Reading QR code...


QR code reading complete.



Extracting features: 100%|██████████| 41/41 [00:10<00:00,  4.01it/s]
Extracting features: 0it [00:00, ?it/s]



Image 2 of 2 processed.
Done.


20